In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
project_path = os.getenv("PROJECT_PATH")

In [2]:
%run "{project_path}\llm_custom_apps\llm_apps\oh_sheet_its_spark\utils.ipynb"

In [ ]:
# Function to format the messages properly
def update_chat_history(messages, new_message, sender):
    # Ensure each message is added as a list with two elements
    messages.append([sender, new_message])
    return messages

# Function to generate assistant's responses (dummy implementation for example purposes)
def generate_responses_from_inf(token, chat_history, model, max_tokens):
    # Dummy response generation based on chat history (replace with your actual logic)
    assistant_response = "This is a response from the assistant."
    return assistant_response

# Gradio UI
with gr.Blocks(css="""
    .title {text-align: center; color: #4a4a4a; font-size: 24px; margin-top: 20px;}
    #upload_button {margin-bottom: 20px;}
    #reset_button {background-color: #f39c12; color: white; margin-top: 20px;}
    #send_button {background-color: #3498db; color: white;}
    body {background-color: #f5f7fb;}  /* Light background color */
    .gradio-container {background-color: #ffffff;}  /* White background for the main container */
    .chatbot {background-color: #eaf0f6;}  /* Light blue for the chatbot */
    .gr-chatbot-container {background-color: #eaf0f6;} /* Background for the chat window */
    .gr-button {font-weight: bold; padding: 10px;} /* Make buttons bolder for better UX */
""") as app:
    gr.Markdown("# Excel-to-PySpark Chatbot", elem_id="title")

    with gr.Row():
        with gr.Column(scale=1, min_width=200):
            upload_button = gr.File(label="Upload Excel File", file_types=[".xlsx"], elem_id="upload_button")
            reset_button = gr.Button("Retry", elem_id="reset_button")
            error_message = gr.Textbox(label="Error", visible=False, interactive=False, elem_id="error_box")
            user_input = gr.Textbox(label="Your Message", placeholder="Type your message here...", elem_id="user_input")
            send_button = gr.Button("Send", elem_id="send_button")

        with gr.Column(scale=3):
            chatbot = gr.Chatbot(elem_id="chat_window", label="Assistant", height=400)

    # Function for handling initial file upload (no initial system/user prompt)
    def on_upload(file):
        return [], None  # Just return an empty chat for now, no system/user prompts

    # Function to handle sending messages
    def on_send(user_message, chat_history):
        assistant_response = generate_responses_from_inf("hf_token", chat_history, "Qwen/Qwen2.5-Coder-32B-Instruct", 5000)
        # Append the user message and assistant response as proper format
        chat_history = update_chat_history(chat_history, user_message, "user")
        chat_history = update_chat_history(chat_history, assistant_response, "assistant")
        return chat_history

    # Function to reset the conversation
    def on_reset():
        return [], None  # Reset chat history and no error message

    # Event Bindings
    upload_button.upload(on_upload, inputs=[upload_button], outputs=[chatbot])
    send_button.click(on_send, inputs=[user_input, chatbot], outputs=[chatbot])
    reset_button.click(on_reset, inputs=[], outputs=[chatbot, error_message])

d:\gitprojects\llm_custom_apps\.venv\Lib\site-packages\gradio\components\chatbot.py:242: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


In [6]:
# Launch the app
app.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "d:\gitprojects\llm_custom_apps\.venv\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\gitprojects\llm_custom_apps\.venv\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\gitprojects\llm_custom_apps\.venv\Lib\site-packages\gradio\blocks.py", line 2057, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\gitprojects\llm_custom_apps\.venv\Lib\site-packages\gradio\blocks.py", line 1863, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\gitprojects\llm_custom_apps\.v